# Preprocessing

In [1]:
import os
import re
from nltk.tokenize import word_tokenize
import numpy as np
#from gensim.models import Word2Vec

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
diacritics_dict = {'َ': 0, 'ً': 1, 'ُ': 2, 'ٌ': 3, 'ِ': 4, 'ٍ': 5, 'ْ': 6, 'ّ': 7, 'َّ': 8, 'ًّ': 9, 'ُّ': 10, 'ٌّ': 11, 'ِّ': 12, 'ٍّ': 13, '': 14}

In [4]:
# Define a list of allowed characters (Arabic characters with diacritics)
allowed_characters = [
     'ذ', 'ت', 'ع', 'س', 'خ', 'ئ', 'ط', 'ث', 'ص', 'ح', 'إ', 'غ', 'ا', 'ب', 'ج', 'ز', 'آ', 'ر',
     'ش', 'ي', 'ض', 'ه', 'ق', 'ء', 'ن', 'د', 'ة', 'ف', 'ؤ', 'ظ', 'ل', 'م', 'ك', 'ى', 'أ', 'و',
     'ّ', 'َ', 'ً', 'ٍ', 'ْ', 'ِ', 'ُ', 'ٌ'
 ]
def clean_text(text):

    # Remove characters not in the allowed list
    cleaned_text = ''.join(char if char in allowed_characters else ' ' for char in text)

    # Collapse consecutive spaces into a single space
    cleaned_text = ' '.join(cleaned_text.split())

    return cleaned_text

In [5]:
def tokenize_text(text):

    # Tokenize the text
    tokens = word_tokenize(text)

    return tokens

In [6]:
def preprocess_text(text):

    # Clean the text
    cleaned_text = clean_text(text)

    # Tokenize the cleaned text
    tokens = tokenize_text(cleaned_text)

    return tokens

In [7]:
# Extract letters in X and corresponding Diacritics in Y and same for words
def extract_arabic_and_Diacritics(arabic_tokens):
    # List of characters without Diacritics
    characters = []
    # List of words without Diacritics
    words = []
    # List of Diacritics corresponding to each character in characters list
    Diacritics_characters = []
    # List of Diacritics corresponding to each word in words list
    Diacritics_words = []

    # All Diacritics letters in Arabic
    Diacritics_set = ('َ', 'ُ', 'ِ', 'ّ', 'ْ', 'ٌ', 'ً', 'ٍ')

    # For each word with Diacritics
    for token in arabic_tokens:
        token_diacritic = []
        # Will be concatinated with letters without Diacritics
        word_without_Diacritics = ''
        # Check if the last letter in the word is Diacritics ==> Add it to Diacritics_words
        # Look at 2 letters as the diacritic may be 2 letters
        if token[-1] in Diacritics_set:
            token_diacritic.append(token[-1])
        # If the last character isn't Diacritics ==> Add None to Diacritics_words
        else:
            token_diacritic.append(None)
        if(len(token)>=2):
            if token[-2] in Diacritics_set:
                token_diacritic.append(token[-2])
            # If the last character isn't Diacritics ==> Add None to Diacritics_words
            else:
                token_diacritic.append(None)
        Diacritics_words.append(token_diacritic)
        # Loop over all characters of this word
        for char in token:
            # Check it isn't Diacritics (It is Arabic characher)
            if char not in Diacritics_set:
                # Add the character without Diacritics to characters list
                characters.append(char)
                # Concatinate the character without Diacritics to the word
                word_without_Diacritics += char

                # Extract Diacritics (it's the character after the Arabic letter)
                Diacritics_index = token.index(char) + 1
                # A list because it can be more than one Diacritics characters
                Diacritics = []
                # Check it is a Diacritics character
                if Diacritics_index < len(token) and token[Diacritics_index] in Diacritics_set:
                    Diacritics.append(token[Diacritics_index])
                    Diacritics_index +=1
                    # Diacritics may be 2 characters if it is (شدة + فتحة/ضمة/كسرة)
                    # Check for the character after the Diacritics character
                    if Diacritics_index < len(token) and token[Diacritics_index] in Diacritics_set:
                        Diacritics.append(token[Diacritics_index])
                    else:
                        # The second Diacritics character is None
                        Diacritics.append(None)
                else:
                    # No Diacritics
                    Diacritics.extend([None, None])
                # Add the Diacritics of the character to the list item corresponding to the character in characters list
                Diacritics_characters.append(Diacritics)
        # Add the word whithout Diacritics to the words list
        words.append(word_without_Diacritics)


    return characters, Diacritics_characters, words, Diacritics_words

In [8]:
def read_dataset(dataset_path):
    #dataset_path = r'/home/donia/Desktop/college/NLP/project/NLP-Arabic-Diacritics-Detector/dataset/train.txt'
    # Get the path to the current script's directory
    current_file_path = os.getcwd()

    # Construct the full path to the image
    dataset_path = os.path.join(current_file_path, dataset_path)

    with open(dataset_path, 'r', encoding='utf-8') as file:
        lines = [line.strip() for line in file.readlines()]

    # Split each line on dots to minimize length of line
    split_lines = [line.split('.') for line in lines]
    # Flatten the lines after splitting so each splitted part be a separate line
    # Remove empty lines
    flat_split_lines = [part for line in split_lines for part in line if part.strip()]

    #Lists to contain all the training data and its labels
    X_letters_list =[]
    Y_letters_list =[]
    X_words_list =[]
    Y_words_list =[]


    tokenized_lines =[]


    # Test the functions on the first 10 lines
    for line in flat_split_lines:
        cleaned_line = clean_text(line)
        removed = set(line) - set(cleaned_line)
        tokenized_line = tokenize_text(cleaned_line)

        X_letters, Y_letters, X_words, Y_words = extract_arabic_and_Diacritics(tokenized_line)
        X_letters_list.append(X_letters)
        Y_letters_list.append(Y_letters)
        X_words_list.append(X_words)
        Y_words_list.append(Y_words)
        tokenized_lines.append(tokenized_line)

    return X_letters_list, Y_letters_list, X_words_list, Y_words_list

In [9]:
def extract_letters_classes(Y_letters_list):
    letters_diacritics_classes = []
    for line in Y_letters_list:
        line_letters_diacritics_classes=[]
        for diacritic in line:
            diacritic_string = ""
            if(diacritic[0]):
                diacritic_string = diacritic[0]
            if(diacritic[1]):
                diacritic_string+=diacritic[1]
            line_letters_diacritics_classes.append(diacritics_dict[diacritic_string])
        letters_diacritics_classes.append(line_letters_diacritics_classes)
    return letters_diacritics_classes

In [10]:
# preprocessing training dataset
X_letters_list, Y_letters_list, X_words_list, Y_words_list = read_dataset("train.txt")

In [11]:
letters_diacritics_classes = extract_letters_classes(Y_letters_list)

In [12]:
# preprocessing validation dataset
val_X_letters_list, val_Y_letters_list, val_X_words_list, val_Y_words_list = read_dataset("val.txt")

In [13]:
val_letters_diacritics_classes = extract_letters_classes(val_Y_letters_list)

# One hot encoding

In [14]:
def generate_id_mapping(characters):
    char_to_id = {char: idx for idx, char in enumerate(characters)}
    id_to_char = {idx: char for idx, char in enumerate(characters)}
    return char_to_id, id_to_char

def generate_char_one_hot_vector(char_id, char_vector_size=36):
    char_vector = np.zeros(char_vector_size)
    char_vector[char_id] = 1
    return char_vector

Generate one hot encoding for train dataset

In [15]:
characters = [
     'ذ', 'ت', 'ع', 'س', 'خ', 'ئ', 'ط', 'ث', 'ص', 'ح', 'إ', 'غ', 'ا', 'ب', 'ج', 'ز', 'آ', 'ر',
     'ش', 'ي', 'ض', 'ه', 'ق', 'ء', 'ن', 'د', 'ة', 'ف', 'ؤ', 'ظ', 'ل', 'م', 'ك', 'ى', 'أ', 'و',

 ]

In [16]:
char_to_id, _ = generate_id_mapping(characters)

In [17]:
train_characters = [char for sequence in X_letters_list for char in sequence]

X = [generate_char_one_hot_vector(char_to_id[char]) for char in train_characters ]

In [18]:
del X_letters_list
del train_characters
del Y_letters_list
del X_words_list
del Y_words_list
del allowed_characters
del diacritics_dict

In [19]:
Y = [item for sublist in letters_diacritics_classes for item in sublist]

Generate one hot encoding for validation dataset

In [20]:
characters = [char for sequence in val_X_letters_list for char in sequence]

val_X = [generate_char_one_hot_vector(char_to_id[char]) for char in characters ]

val_Y = [item for sublist in val_letters_diacritics_classes for item in sublist]

In [21]:
x_train = np.array(X)
del X

In [22]:
y_train = np.array(Y)
del Y

In [23]:
x_test = np.array(val_X)
del val_X

In [24]:
y_test = np.array(val_Y)
del val_Y

In [25]:
print(y_test.shape)
print(x_test.shape)
print(x_train.shape)
print(y_train.shape)

(421099,)
(421099, 36)
(8351478, 36)
(8351478,)


# Bidirectional LSTM


In [25]:
# Reshape input data for LSTM
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

In [26]:
from tensorflow import keras
from tensorflow.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM, BatchNormalization
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_uniform
import tensorflow as tf

In [27]:
dropout = 0.5  # Adjusted dropout rate
window_size = 256
input_shape = (1, 36)
num_classes = 15
batch_size = 256

In [28]:
model = Sequential()

model.add(Bidirectional(LSTM(window_size,
                             return_sequences=True,
                             input_shape=input_shape,
                             kernel_initializer=glorot_uniform())))  # Apply Glorot Uniform initializer
model.add(Dropout(rate=dropout))

model.add(Bidirectional(LSTM(window_size, return_sequences=True, kernel_initializer=glorot_uniform())))
model.add(Dropout(rate=dropout))
model.add(Activation('relu'))  # Add ReLU activation after LSTM

model.add(Bidirectional(LSTM(window_size, return_sequences=False, kernel_initializer=glorot_uniform())))
model.add(Dense(units=num_classes, activation='softmax'))

In [ ]:
optimizer = Adam(learning_rate=0.001)

y_train = tf.cast(y_train, tf.int32)
y_train = tf.one_hot(y_train, num_classes)

y_test = tf.cast(y_test, tf.int32)
y_test = tf.one_hot(y_test, num_classes)

model.compile(loss='categorical_crossentropy', optimizer=optimizer , metrics=['accuracy'])

history = model.fit(
    x_train,
    y_train,
    epochs=50,
    batch_size=batch_size,
    shuffle=False,
    validation_data=(x_test, y_test)
)

Epoch 1/50
32623/32623 [==============================] - 412s 12ms/step - loss: 1.2824 - accuracy: 0.5176 - val_loss: 1.2808 - val_accuracy: 0.5175
Epoch 2/50
32623/32623 [==============================] - 392s 12ms/step - loss: 1.2768 - accuracy: 0.5183 - val_loss: 1.2806 - val_accuracy: 0.5156
Epoch 3/50
32623/32623 [==============================] - 389s 12ms/step - loss: 1.2767 - accuracy: 0.5184 - val_loss: 1.2805 - val_accuracy: 0.5175
Epoch 4/50
32623/32623 [==============================] - 389s 12ms/step - loss: 1.2766 - accuracy: 0.5184 - val_loss: 1.2804 - val_accuracy: 0.5156
Epoch 5/50
32623/32623 [==============================] - 388s 12ms/step - loss: 1.2766 - accuracy: 0.5184 - val_loss: 1.2804 - val_accuracy: 0.5156
Epoch 6/50
32623/32623 [==============================] - 389s 12ms/step - loss: 1.2766 - accuracy: 0.5184 - val_loss: 1.2803 - val_accuracy: 0.5156
Epoch 7/50
32623/32623 [==============================] - 396s 12ms/step - loss: 1.2765 - accuracy: 0.5184